In [153]:
import tensorflow as tf
import transformers
import data

In [154]:
import os
os.environ['TF_GPU_ALLOCATOR']="cuda_malloc_async"

In [155]:
model_name = "FacebookAI/roberta-base"
tokenizer = transformers.RobertaTokenizer.from_pretrained(model_name)
x_train, y_train, x_test, y_test = data.load_data("soraby") 

In [156]:
def tokenize_function(texts):
    encodings = tokenizer(
        texts, 
        truncation=True, 
        padding=True, 
        max_length=128, 
        return_tensors="tf"
    )
    return encodings
    # return {key: encodings[key] for key in encodings}, labels

In [157]:
train_enc = tokenize_function(x_train)
test_enc = tokenize_function(x_test)

In [158]:
type(train_enc)

transformers.tokenization_utils_base.BatchEncoding

In [159]:
dataset = tf.data.Dataset.from_tensor_slices((dict(train_enc), y_train))

In [160]:
batch_size = 16
dataset = dataset.shuffle(len(y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [161]:
roberta_model = transformers.TFRobertaModel.from_pretrained(model_name, num_labels=2)
input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
roberta_outputs = roberta_model(input_ids, attention_mask=attention_mask)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

ValueError: Exception encountered when calling layer 'tf_roberta_model_16' (type TFRobertaModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_roberta_model_16' (type TFRobertaModel):
  • input_ids=<KerasTensor shape=(None, 128), dtype=int32, sparse=False, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 128), dtype=int32, sparse=False, name=attention_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [ ]:
x1 = tf.keras.layers.Dropout(0.1)(roberta_model)
x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
x1 = tf.keras.layers.LeakyReLU()(x1)
x1 = tf.keras.layers.Dense(1)(x1)
x1 = tf.keras.layers.Flatten()(x1)
output = tf.keras.layers.Activation('softmax')(x1)

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: <transformers.models.roberta.modeling_tf_roberta.TFRobertaModel object at 0x7b008fda1c10> (of type <class 'transformers.models.roberta.modeling_tf_roberta.TFRobertaModel'>)

In [ ]:
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

NameError: name 'output' is not defined

In [ ]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
model.fit(dataset, epochs=3)

Epoch 1/3


ValueError: in user code:

    File "/home/dayan/projects/sarc2/.conda/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/dayan/projects/sarc2/.conda/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/dayan/projects/sarc2/.conda/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/home/dayan/projects/sarc2/.conda/lib/python3.11/site-packages/transformers/modeling_tf_utils.py", line 1706, in train_step
        loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)
    File "/home/dayan/projects/sarc2/.conda/lib/python3.11/site-packages/tf_keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, 